In [ ]:
import random
import os
import json
import torch
import optuna
import numpy as np
import math
from collections import Counter
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedShuffleSplit
from sentence_transformers import SentenceTransformer
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import Callback
from torchmetrics.classification import MulticlassAccuracy

# === SEED ===
SEED = 42
def seed_everything(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    pl.seed_everything(seed, workers=True)

# Ensure it's called!
seed_everything(SEED)

# === CONFIG ===
DATASET_PATH = "data/us_gaap_multilabel_training_data.json"
MODEL_NAME = "BAAI/bge-large-en-v1.5"
OUTPUT_PATH = "data/fine_tuned_gaap_classifier"
os.makedirs(OUTPUT_PATH, exist_ok=True)
OPTUNA_DB_PATH = os.path.join(OUTPUT_PATH, "optuna_study.db")
BATCH_SIZE = 16
EPOCHS = 100
PATIENCE = 3
VALIDATION_SPLIT = 0.2

device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

with open(DATASET_PATH, "r") as f:
    data = json.load(f)

def label_hash(entry):
    return f"{entry['labels']['statement_type']}_{entry['labels']['balance']}_{entry['labels']['period_type']}"

def get_split():
    label_hashes = [label_hash(d) for d in data]
    hash_counts = Counter(label_hashes)
    common_data, common_labels, rare_data = [], [], []
    for i, lh in enumerate(label_hashes):
        if hash_counts[lh] >= 2:
            common_data.append(data[i])
            common_labels.append(lh)
        else:
            rare_data.append(data[i])
    splitter = StratifiedShuffleSplit(n_splits=1, test_size=VALIDATION_SPLIT, random_state=42)
    train_idx, val_idx = next(splitter.split(common_data, common_labels))
    train_data = [common_data[i] for i in train_idx]
    val_data = [common_data[i] for i in val_idx]
    train_data.extend(rare_data)
    return train_data, val_data

class MultiLabelDataset(Dataset):
    def __init__(self, data):
        self.samples = [
            (d["text"], torch.tensor([
                d["labels"]["statement_type"],
                d["labels"]["balance"],
                d["labels"]["period_type"]
            ], dtype=torch.long))
            for d in data
        ]
    def __len__(self):
        return len(self.samples)
    def __getitem__(self, idx):
        return self.samples[idx]

def collate_fn(batch):
    texts, labels = zip(*batch)
    return list(texts), torch.stack(labels)

class AttentionLayer(torch.nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int = 128):
        super().__init__()
        self.attn = torch.nn.Sequential(
            torch.nn.Linear(input_dim, hidden_dim),
            torch.nn.Tanh(),
            torch.nn.Linear(hidden_dim, 1)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Apply attention over a batch of embeddings.
        x: (batch_size, input_dim)
        returns: (batch_size, input_dim)
        """
        # Simulate sequence: [batch_size, 1, dim]
        x = x.unsqueeze(1)
        weights = torch.softmax(self.attn(x), dim=1)  # (batch_size, 1, 1)
        attended = torch.sum(weights * x, dim=1)      # (batch_size, dim)
        return attended


class GAAPClassifier(pl.LightningModule):
    def __init__(self, lr, weight_decay, model_name):
        super().__init__()
        self.lr = lr
        self.weight_decay = weight_decay
        self.model_name = model_name
        self.encoder = SentenceTransformer(model_name, device=device)
        dim = self.encoder.get_sentence_embedding_dimension()

        self.attn = AttentionLayer(dim)

        self.head_st = torch.nn.Linear(dim, 3) # Dependent on number of classes
        self.head_bal = torch.nn.Linear(dim, 3) # Dependent on number of classes
        self.head_pt = torch.nn.Linear(dim, 3) # Dependent on number of classes
        self.loss_fn = torch.nn.CrossEntropyLoss()
        self.save_hyperparameters()

        self.acc_st = MulticlassAccuracy(num_classes=3, average='micro')
        self.acc_bal = MulticlassAccuracy(num_classes=3, average='micro')
        self.acc_pt = MulticlassAccuracy(num_classes=3, average='micro')


    def forward(self, texts):
        with torch.no_grad():
            embeddings = self.encoder.encode(
                texts, convert_to_tensor=True, device=device
            )

        attended = self.attn(embeddings)
        out_st = self.head_st(attended)
        out_bal = self.head_bal(attended)
        out_pt = self.head_pt(attended)

        return out_st, out_bal, out_pt

    def compute_loss(self, outputs, labels):
        out_st, out_bal, out_pt = outputs
        loss_0 = self.loss_fn(out_st, labels[:, 0])
        loss_1 = self.loss_fn(out_bal, labels[:, 1])
        loss_2 = self.loss_fn(out_pt, labels[:, 2])
        return loss_0 + loss_1 + loss_2, (loss_0, loss_1, loss_2)

    def training_step(self, batch, batch_idx):
        texts, labels = batch
        outputs = self(texts)
        loss, (l0, l1, l2) = self.compute_loss(outputs, labels)
        self.log("train/loss", loss, prog_bar=True)
        self.log("train/loss_statement_type", l0)
        self.log("train/loss_balance", l1)
        self.log("train/loss_period_type", l2)

        pred_st = torch.argmax(outputs[0], dim=1)
        pred_bal = torch.argmax(outputs[1], dim=1)
        pred_pt = torch.argmax(outputs[2], dim=1)

        self.log("train/acc_statement_type", self.acc_st(pred_st, labels[:, 0]))
        self.log("train/acc_balance", self.acc_bal(pred_bal, labels[:, 1]))
        self.log("train/acc_period_type", self.acc_pt(pred_pt, labels[:, 2]))

        return loss

    def validation_step(self, batch, batch_idx):
        texts, labels = batch
        outputs = self(texts)
        loss, (l0, l1, l2) = self.compute_loss(outputs, labels)
        self.log("val/loss", loss, prog_bar=True)
        self.log("val/loss_statement_type", l0)
        self.log("val/loss_balance", l1)
        self.log("val/loss_period_type", l2)

        pred_st = torch.argmax(outputs[0], dim=1)
        pred_bal = torch.argmax(outputs[1], dim=1)
        pred_pt = torch.argmax(outputs[2], dim=1)

        self.log("val/acc_statement_type", self.acc_st(pred_st, labels[:, 0]))
        self.log("val/acc_balance", self.acc_bal(pred_bal, labels[:, 1]))
        self.log("val/acc_period_type", self.acc_pt(pred_pt, labels[:, 2]))

        return loss

    def configure_optimizers(self):
        params = list(self.head_st.parameters()) + \
                 list(self.head_bal.parameters()) + \
                 list(self.head_pt.parameters())
        return torch.optim.AdamW(
            params,
            lr=self.lr,
            weight_decay=self.weight_decay
        )

class SafePruningCallback(Callback):
    def __init__(self, trial, monitor: str = "val/loss"):
        super().__init__()
        self.trial = trial
        self.monitor = monitor

    def on_validation_end(self, trainer, pl_module):
        current = trainer.callback_metrics.get(self.monitor)
        if current is None:
            return
        self.trial.report(current, step=trainer.global_step)
        if self.trial.should_prune():
            raise optuna.TrialPruned()

def objective(trial):
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
    weight_decay = trial.suggest_float("weight_decay", 0.0, 0.1)

    train_data, val_data = get_split()
    train_loader = DataLoader(MultiLabelDataset(train_data),
                              batch_size=BATCH_SIZE,
                              shuffle=True,
                              collate_fn=collate_fn)
    val_loader = DataLoader(MultiLabelDataset(val_data),
                            batch_size=BATCH_SIZE,
                            shuffle=False,
                            collate_fn=collate_fn)

    model = GAAPClassifier(lr=lr, weight_decay=weight_decay,
                           model_name=MODEL_NAME)

    log_dir = os.path.join(OUTPUT_PATH, f"trial_{trial.number}")
    logger = TensorBoardLogger(save_dir=log_dir, name="logs")

    early_stop = EarlyStopping(monitor="val/loss", patience=PATIENCE, mode="min")
    checkpoint_cb = ModelCheckpoint(
        dirpath=os.path.join(log_dir, "checkpoints"),
        filename="best",
        monitor="val/loss",
        mode="min",
        save_top_k=1,
        save_weights_only=True
    )

    trainer = pl.Trainer(
        logger=logger,
        max_epochs=EPOCHS,
        enable_checkpointing=True,
        callbacks=[
            early_stop,
            checkpoint_cb,
            SafePruningCallback(trial, monitor="val/loss")
        ],
        log_every_n_steps=1,
        accelerator="auto",
        devices=1
    )

    trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)
    print("Best checkpoint:", checkpoint_cb.best_model_path)
    return trainer.callback_metrics["val/loss"].item()


In [ ]:
study = optuna.create_study(
     direction="minimize",
     study_name="gaap_tuning",
     storage=f"sqlite:///{OPTUNA_DB_PATH}",
     load_if_exists=True
)
study.optimize(objective, n_trials=50)
print("Best params:", study.best_params)

print("Best trial:")
best_trial = study.best_trial
print("  Value: %.6f" % best_trial.value)
print("  Params:")
for k, v in best_trial.params.items():
    print("    %s: %s" % (k, v))

